In [ ]:
# default_exp local_runner

In [55]:
#export
import IPython.core
import json
import os.path
import re
import ipykernel
import requests

#try:  # Python 3
#    from urllib.parse import urljoin
#except ImportError:  # Python 2
#    from urlparse import urljoin

# Alternative that works for both Python 2 and 3:
from requests.compat import urljoin

try:  # Python 3 (see Edit2 below for why this may not work in Python 2)
    from notebook.notebookapp import list_running_servers
except ImportError:  # Python 2
    import warnings
    from IPython.utils.shimmodule import ShimWarning
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=ShimWarning)
        from IPython.html.notebookapp import list_running_servers


def get_notebook_path():
    """
    Return the full path of the jupyter notebook.
    !!! Doesn't work from VS code.
    """
    kernel_id = re.search('kernel-(.*).json',
                          ipykernel.connect.get_connection_file()).group(1)
    servers = list_running_servers()
    for ss in servers:
        try:
            response = requests.get(urljoin(ss['url'], 'api/sessions'),
                                    params={'token': ss.get('token', '')}, timeout=1)
            for nn in json.loads(response.text):
                if nn['kernel']['id'] == kernel_id:
                    relative_path = nn['notebook']['path']
                    return os.path.join(ss['notebook_dir'], relative_path)
        except Exception as e:
            pass


In [54]:
#hide
print(get_notebook_path())

/usr/local/google/home/alekseyv/vlasenkoalexey/gcp_runner/run_local.ipynb


In [62]:
#export
import nbdev.export

def find_default_export_for_notebook(notebook_path):
    nb = nbdev.export.read_nb(notebook_path)
    default = nbdev.export.find_default_export(nb['cells'])
    return default

def get_module_name():
    return find_default_export_for_notebook(get_notebook_path())


In [41]:
#hide
print(get_module_name())


run_local


In [26]:
#hide
nbdev.export.get_nbdev_module().__name__

'gcp_runner._nbdev'

In [35]:
#export
import nbdev.imports

def get_package_name():
    # see logic in https://github.com/fastai/nbdev/blob/master/nbdev/export.py
    return str(nbdev.imports.Config().lib_path).split('/')[-1]

In [36]:
get_package_name()

'gcp_runner'

In [168]:
#export

import os
import subprocess
import io
from threading import Thread
from queue import Queue 
from nbdev.export import *


def _reader(pipe, queue, pipe_name):
    try:
        with pipe:
            for line in iter(pipe.readline, b''):
                queue.put((pipe_name, line))
    finally:
        queue.put(None)
        
def _get_run_python_args(func, **kwargs):
    package_name = get_package_name()
    module_name = get_module_name()
    function_name = func.__name__
    args = ['python3', 
         '-u',
         '-m', 
         'gcp_runner.entry_point',
         '--module-name=%s' % package_name + '.' + module_name,
         '--function-name=%s' % function_name]
    return args
        
def run_python(func, **kwargs):
    args = _get_run_python_args(func, **kwargs)
    run_process(args, func_name)

def run_process(args, **kwargs):
    proc = subprocess.Popen(
        args, 
        stdout=subprocess.PIPE, 
        stderr=subprocess.PIPE, 
        bufsize=1)
    
    #(stdout, stderr) = proc.communicate(timeout=15)
    
    q = Queue()
    Thread(target=_reader, args=[proc.stdout, q, 'stdout']).start()
    Thread(target=_reader, args=[proc.stderr, q, 'stderr']).start()
    #for _ in range(2):
    for source, line in iter(q.get, None):
        line = line.decode('utf-8').rstrip('\r\n')
        if source == 'stderr':
            print("\x1b[31m{}\x1b[0m".format(line))
        else:
            print(line)
    
    return proc.poll()

In [181]:
if run_process(['ls']):
    print('err')

00_core.ipynb
ai_platform_constants.ipynb
ai_platform_runner.ipynb
CONTRIBUTING.md
Dockerfile
docs
entry_point.ipynb
gcp_runner
index.ipynb
LICENSE
local_runner.ipynb
MANIFEST.in
README.md
sample_code.ipynb
settings.ini
setup.py


In [183]:
if 1:
    print('0')

0


In [130]:
#export

def run_docker(func, image_uri, docker_args=None, **kwargs):
    package_name = get_package_name()
    args = ['docker', 'run', '-v', 
            os.getcwd() + '/%s:/%s' % (package_name, package_name)]
    if docker_args:
        args.extend(docker_args)
    args.append(image_uri)
    run_python_args = _get_run_python_args(func, **kwargs)
    args.extend(run_python_args)
    print('running in docker:')
    print(' '.join(args))
    run_process(args) 
    

In [155]:
#export

#IMPORTANT: we'll need to either copy entry_point.py to package,
# or tell user to declare it and call into similar entry_point logic manually.
#
# !!! Doesn't work because of https://github.com/GoogleCloudPlatform/cloudml-samples/issues/476
def run_on_ai_platform(func, job_dir):
    package_name = get_package_name()
    module_name = get_module_name()
    function_name = func.__name__
    args = ['gcloud', 'ai-platform', 'local', 'train',
           "--job-dir=%s" % job_dir,
           "--module-name=%s.entry_point" % package_name,
           "--package-path=%s/%s " % (os.getcwd(), package_name),
           "--",
           "--job-dir=%s" % job_dir, 
           "--module-name=%s.%s" % (package_name, module_name),
           "--function-name=%s" % function_name]
    print('running training job using local Cloud AI:')
    print(' '.join(args))
    run_process(args)
    

In [160]:
from nbdev.export import *
notebook2script()


Converted 00_core.ipynb.
Converted entry_point.ipynb.
Converted index.ipynb.
Converted local_runner.ipynb.
Converted sample_code.ipynb.
